In [16]:
import numpy as np
import os

if os.getcwd()[-9:]=="notebooks":
    os.chdir("..")

import sMZI as itf

In [17]:
def print_matrix(M: np.ndarray, prec: int=2):
    """
    Function to print a given matrix in a nice way.
    
    Parameters
    ------
    M : matrix to print
    prec : floating point precision
    """
    for row in M:
        print(f"{np.array2string(row, precision=prec ,formatter={'float': lambda row: f'{row:.2f}'},separator=', ')}")
    print('\n')

def print_data_ext_ps(V1: np.complex_, V2: np.complex_):
    """
    Function to print data on the effect of the external phaseshift
    that's used to match the phases of two given elements.
    
    Parameters
    ------
    V1 : element of the auxillary matrix `V`
    V2 : subsequent element of **V1**
    
    ---
    
    Further description:
    ---
    When it comes to the effect of the external phaseshift  `P`:
    
    - for **even** diagonals ( j=2,4... ):
    >> ``V1 = V[x,y]`` ,  ``V2 = V[x-1,y]``
    
    - for **odd** diagonals ( j=1,3... ):
    >> ``V1 = V[x,y]`` ,  ``V2 = V[x,y+1]``
    
    ---
    When it comes to the effect of `exp(i*summa)` in `M`:
    
    - for **even** diagonals ( j=2,4... ):
    >> ``V1 = V[x-1,y-1]`` ,  ``V2 = V[x-1,y]``
    
    - for **odd** diagonals ( j=1,3... ):
    >> ``V1 = V[x+1,y+1]`` ,  ``V2 = V[x,y+1]``
    
    """
    
    eq_stat = (np.angle(V1) == np.angle(V2))
    eq_str = "\nThey are matching!"
    neq_str = "\nThey are NOT matching!\n"

    print('Affected elements:\t',
        "{:.2f}, {:.2f}".format(V1, V2))
    print('Corresponding angles:\t',
        "{:.2f}, {:.2f}".format(np.angle(V1), np.angle(V2)))

    if eq_stat:
        print(eq_str)
    else:
        print(neq_str)
        print('Full length of angles:\n',
            "{}, {}".format(np.angle(V1), np.angle(V2)))
        print('\nTheir difference:\t',
            "{}".format(np.angle(V1) - np.angle(V2)))

In [18]:
U = itf.random_unitary(5)

# Test of algorithm steps

## Odd diagonals

### Finding external phaseshifts
They match the given elements' phases

In [19]:
V = np.conjugate(U.T)
# odd diags: 1,3,5...
m = U.shape[0]
j = 1

x = m-1
y = j-1

P = itf.external_ps(m, j, V[x,y], V[x,y+1])
# print('old V:\n')
# print_matrix(V)
V = np.matmul(V,P)
# print("NOTE: {}. column was affected!".format(j+1))
# print('new V:\n')
# print_matrix(V)

print_data_ext_ps(V[x,y],V[x,y+1])

Affected elements:	 0.41-0.19j, 0.41-0.19j
Corresponding angles:	 -0.44, -0.44

They are NOT matching!

Full length of angles:
 -0.4358544045762982, -0.43585440457629826

Their difference:	 5.551115123125783e-17


### Finding internal phaseshifts
$\delta$ and $\sum$

In [20]:
# looking at odd diagonal version!

# minus sign !
delta = itf.custom_arctan(V[x,y+1], V[x,y])

print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

# if k == j: summ = 0
summ = 0

# might need to change into 'k' dependence
modes = [y, y+1]    # initial mode-pairs    NOTE: need to update it to x,y dependence
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ)

print('\nBlock of M:\n')
print_matrix(M[y:y+2,y:y+2])


# print('old V:\n')
# print_matrix(V)
V = np.matmul(V,M)
# print("NOTE: {}. and {}. column were affected!".format(j,j+1))
# print('new V:\n')
# print_matrix(V)

delta:	-0.78+0.00j	 -> real: False
angle:	180.0°

Block of M:

[-0.7 +2.00e-17j,  0.71+1.97e-17j]
[0.71+1.97e-17j, 0.7 -2.00e-17j]




In [21]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
# print('Elements affected by e^(i*summ)')
# print_data_ext_ps(V[x-1,y], V[x-1,y+1])

Nulled element:	Re(0.00) Im(-0.00)
abs: 0.00, angle: -38.90°



## Even diagonals

### Finding external phaseshifts
They match the given elements' phases

In [22]:
# V = np.conjugate(U.T)
# even diags: 2,4,6...
m = U.shape[0]
j = 2

x = m-j
y = 0

P = itf.external_ps(m, j, V[x,y], V[x-1,y])
# print('old V:\n')
# print_matrix(V)
V = np.matmul(P,V)
# print("NOTE: {}. row was affected!".format(j+1))
# print('new V:\n')
# print_matrix(V)

print_data_ext_ps(V[x,y],V[x-1,y])

Affected elements:	 -0.26-0.48j, -0.05-0.10j
Corresponding angles:	 -2.07, -2.07

They are matching!


### Finding internal phaseshifts
$\delta$ and $\sum$

In [23]:
# looking at odd diagonal version!

# minus sign !
delta = itf.custom_arctan(-V[x-1,y], V[x,y])

print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

summ = np.angle(V[x+1,y+1]) - np.angle(V[x-1,y+1]*np.cos(delta) - V[x,y+1]*np.sin(delta))

# print('\nangles to get equal by summa:\n',
#       np.angle(V[x+1,y+1]),
#       np.angle(V[x,y+1]*np.cos(delta) - V[x+1,y+1]*np.sin(delta)),'\n',
#       'summa:',summ)

# summ = 0

# might need to change into 'k' dependence
modes = [x-1, x]    # initial mode-pairs    NOTE: need to update it to x,y dependence
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ)

# print('\nBlock of M:\n')
# print_matrix(M[x-1:x+1,x-1:x+1])
# print_matrix(M)


# print('old V:\n')
# print_matrix(V)
V = np.matmul(M,V)
# print("NOTE: {}. and {}. rows were affected!".format(j,j+1))
# print('new V:\n')
# print_matrix(V)

delta:	0.20-0.00j	 -> real: True
angle:	-0.0°


In [24]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
print('Elements affected by e^(i*summ):\n')
print_data_ext_ps(V[x+1,y+1], V[x,y+1])

Nulled element:	Re(0.00) Im(0.00)
abs: 0.00, angle: 40.62°

Elements affected by e^(i*summ):

Affected elements:	 0.58-0.27j, 0.35-0.16j
Corresponding angles:	 -0.44, -0.44

They are NOT matching!

Full length of angles:
 -0.43585440457629826, -0.4358544045762982

Their difference:	 -5.551115123125783e-17


### Continuation on the chosen diagonal
Without correct $\sum$ it's wrong

In [25]:
x += 1
y += 1

delta = itf.custom_arctan(-V[x-1,y], V[x,y])
print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

summ = 0

modes = [x-1, x]     # initial mode-pairs
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ)

# print('\nBlock of M:\n')
# print_matrix(M[x-1:x+1,x-1:x+1])

# print('old V:\n')
# print_matrix(V)
V = np.matmul(M,V)
# print("NOTE: {}. and {}. rows were affected!".format(x,x+1))
# print('new V:\n')
# print_matrix(V)

delta:	0.54+0.00j	 -> real: False
angle:	3.0427994507667812e-15°


In [26]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
# print('Elements affected by e^(i*summ)')
# print_data_ext_ps(V[x-1,y], V[x-1,y+1])

Nulled element:	Re(0.00) Im(0.00)
abs: 0.00, angle: 1.84°



# Test of module
## <center>TODO</center>
* check if $V$ is a diagonal matrix after the decomposition
---
- extend module to save found phases: $\phi$, $\delta$, $\sum$ $\Rightarrow$ $\theta_1$ & $\theta_2$
- check the function in module that recreates the initial unitary matrix based on the decomposed matrices (probably needs work)
- update the drawing function in the module such that it follows the updated **Clement's** `draw()` function but with sMZI:s
- Finally: sweep through the whole module and finalise *comments*, *documentation*, *variables* and implement possible *optimalisations*
---
* optional: `import numba` for faster execution; optimized machine code at runtime

In [27]:
# including imports again in case someone only wishes
# to run this part of the notebook
import numpy as np
import os

if os.getcwd()[-9:]=="notebooks":
    os.chdir("..")

import sMZI as itf

U = itf.random_unitary(5)

In [28]:
final_matrix = itf.square_decomposition(U)

In [32]:
print_matrix(final_matrix)

[ 0.12-1.38j, -0.94-1.67j, -1.47+1.48j,  0.31+0.13j, -0.03+0.37j]
[-0.16+1.04j,  1.09+0.52j,  0.46-0.98j,  0.4 +0.19j, -0.07+0.49j]
[ 2.22e-16-1.18e-16j, -1.15e+00+8.12e-01j,  1.28e+00+5.07e-01j,
  1.83e-01+2.21e-01j, -1.74e-01+2.70e-01j]
[-4.17e-17+5.44e-17j, -2.76e-17+8.83e-17j, -1.38e-01-3.11e-01j,
  3.16e-02+2.34e-01j, -2.39e-01+1.13e-01j]
[-4.22e-17-6.98e-18j, -3.57e-17+7.60e-17j,  7.64e-17+1.09e-17j,
 -1.47e-01+7.30e-01j,  6.63e-01-7.23e-02j]


